In [1]:
from spatialvla.mobilevlm.model.bimanual import load_twinvla_from_singlevla, load_twinvla

[2025-01-07 17:35:21,265] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2025-01-07 17:35:22.826945: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-07 17:35:22.856175: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 17:35:22.856218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 17:35:22.857125: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 17:35:22.862990: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tokenizer, model, image_processor, _ = load_twinvla_from_singlevla('checkpoints/libero_object_octo_full_v2')

You are using a model of type spatialvla to instantiate a model of type twinvla. This is not supported for all configurations of models and can yield errors.


Loading with torch.bfloat16


/home/jellyho/miniconda3/envs/mobilevlm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 32000. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


### Trainig Code Test

In [ ]:
from spatialvla.datasets import RLDSBatchTransform, RLDSDataset
from spatialvla.mobilevlm.model.mobilevlm import load_pretrained_vlm_for_vla, load_vla
from scripts.spatialvla_config import ModelArguments, TrainingArguments
import transformers
from spatialvla.datasets.rlds.utils.data_utils import PaddedCollatorForActionPrediction
from torch.utils.data import DataLoader
from spatialvla.mobilevlm.action_tokenizer import ActionTokenizer
import sys
import torch
import argparse
from PIL import Image
from pathlib import Path
import numpy as np
from spatialvla.mobilevlm.model.mobilevlm import load_vla, load_pretrained_model
from spatialvla.mobilevlm.conversation import conv_templates, SeparatorStyle
from spatialvla.mobilevlm.utils import disable_torch_init, process_images, tokenizer_image_token, KeywordsStoppingCriteria
from spatialvla.mobilevlm.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, Tuple, Type
import numpy as np
import torch
import copy
from PIL import Image
from torch.utils.data import Dataset, IterableDataset
from transformers import PreTrainedTokenizerBase
from transformers import AutoTokenizer, BitsAndBytesConfig
from spatialvla.mobilevlm.utils import disable_torch_init, process_images, tokenizer_image_token, KeywordsStoppingCriteria
# from prismatic.models.backbones.llm.prompting import PromptBuilder
# from prismatic.models.backbones.vision import ImageTransform
from spatialvla.mobilevlm.constants import IGNORE_INDEX, IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from spatialvla.mobilevlm.conversation import conv_templates, SeparatorStyle
from spatialvla.datasets.rlds.utils.data_utils import tree_map
# from prismatic.vla.action_tokenizer import ActionTokenizer
from spatialvla.datasets.rlds import make_interleaved_dataset, make_single_dataset
from spatialvla.datasets.rlds.oxe import OXE_NAMED_MIXTURES, get_oxe_dataset_kwargs_and_weights
from spatialvla.datasets.rlds.utils.data_utils import NormalizationType
from transformers import PreTrainedTokenizerBase

In [ ]:
batch_transform = RLDSBatchTransform(
    tokenizer,
    image_processor,
    use_state_input=True,
    window_size=1,
    future_action_window_size=7,
)
vla_dataset = RLDSDataset(
    data_root_dir='/data1/OXE',
    data_mix='fractal20220817_data',
    batch_transform=batch_transform,
    shuffle_buffer_size=100,
    window_size=1,
    future_action_window_size=7,
    train=True,
    use_state_input=True
)
collator = PaddedCollatorForActionPrediction(
    tokenizer.model_max_length, 
    tokenizer.pad_token_id, 
    padding_side='right', 
    use_state_input=True,
    use_label=False
)
dataloader = DataLoader(
    vla_dataset,
    batch_size=8,
    sampler=None,
    collate_fn=collator,
    num_workers=0,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
)

In [3]:
model.config.training=False
model.config.save_pretrained('checkpoints/twinvla')
model.save_pretrained('checkpoints/twinvla')

In [4]:
tokenizer.save_pretrained('checkpoints/twinvla')

('checkpoints/twinvla/tokenizer_config.json',
 'checkpoints/twinvla/special_tokens_map.json',
 'checkpoints/twinvla/tokenizer.model',
 'checkpoints/twinvla/added_tokens.json')

In [2]:
tokenizer_t, twinvla, image_processor_t, dataset_statistics_t = load_twinvla('checkpoints/twinvla')

Loading with torch.bfloat16


/home/jellyho/miniconda3/envs/mobilevlm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
twinvla

TwinVLA(
  (lm_head): Linear(in_features=2048, out_features=32000, bias=False)
  (action_head): DiffusionActionHead(
    (diffusion_model): ScoreActor(
      (time_preprocess): FourierFeatures()
      (cond_encoder): MLP(
        (mlp): Sequential(
          (0): Linear(in_features=32, out_features=64, bias=True)
          (1): Linear(in_features=64, out_features=32, bias=True)
        )
      )
      (reverse_network): MLPResNet(
        (in_dense): Linear(in_features=295, out_features=256, bias=True)
        (layers): ModuleList(
          (0-2): 3 x MLPResNetBlock(
            (activation): SiLU()
            (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (dense1): Linear(in_features=256, out_features=1024, bias=True)
            (dense2): Linear(in_features=1024, out_features=256, bias=True)
            (dense_residual): Linear(in_features=256, out_features=256, bias=True)
          )
        )
        (out_dense): Linear(in_features=256, out_featur

In [5]:
import torch
torch.distributions.Beta

torch.distributions.beta.Beta

In [7]:
for d in dataloader:
    batch = d
    break
device_id = 0
input_ids=batch['input_ids'].to(device_id)
images=batch['pixel_values'].to(device_id)
attention_mask=batch['attention_mask'].to(device_id)
actions=batch['action'].to(device_id)
use_cache=True
# states=batch['proprio'].to(device_id)
past_key_values = None
# labels = batch['labels'].to(device_id)

/home/jellyho/Bimanual_Imitation/MobileVLM-VLA/spatialvla/datasets/datasets.py:80: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  proprio = torch.Tensor(rlds_batch['observation']['proprio']) if self.use_state_input else None


In [40]:
import mediapy
# print(images[0][0].cpu().transpose(0, 2))
from PIL import Image
print(images[0][0].shape)
i = Image.fromarray(images[0][0].cpu().numpy())
i.save('smapel.png')
img = images[0][0].cpu()
mediapy.show_image(img.transpose(1, 2).transpose(0, 2))

torch.Size([3, 336, 336])


TypeError: Cannot handle this data type: (1, 1, 336), <f2

In [5]:
import torch
if torch.cuda.is_available():
    # Get the current GPU memory usage
    current_memory = torch.cuda.memory_allocated()  # Memory currently allocated by tensors
    reserved_memory = torch.cuda.memory_reserved()  # Total memory reserved by PyTorch
    max_memory = torch.cuda.max_memory_allocated()  # Peak memory usage for allocated tensors

    # Convert to MB for readability
    current_memory_mb = current_memory / (1024 ** 2)
    reserved_memory_mb = reserved_memory / (1024 ** 2)
    max_memory_mb = max_memory / (1024 ** 2)

    print(f"Current Memory Allocated: {current_memory_mb:.2f} MB")
    print(f"Reserved Memory: {reserved_memory_mb:.2f} MB")
    print(f"Peak Memory Allocated: {max_memory_mb:.2f} MB")

Current Memory Allocated: 5739.42 MB
Reserved Memory: 6502.00 MB
Peak Memory Allocated: 6318.32 MB


In [15]:
with torch.autocast('cuda', dtype=torch.bfloat16):
    action  = model.predict_action(
        input_ids=input_ids,
        attention_mask=attention_mask,
        images=images,
        # actions=actions
    )

In [16]:
print(action[0][0][7:])

tensor([ 0.0334, -1.6053,  0.1652, -0.0785,  1.7312,  0.2530,  0.9703],
       device='cuda:0', grad_fn=<SliceBackward0>)


In [17]:
print(action[0][0][:7])

tensor([ 0.1909, -1.6040,  0.1831, -0.0996,  1.5888,  0.5236,  1.0293],
       device='cuda:0', grad_fn=<SliceBackward0>)


In [19]:
actions[0][00]

tensor([ 0.1050, -1.6885,  0.1926, -0.0505,  1.4863,  0.3569,  1.0000],
       device='cuda:0', dtype=torch.float16)